<center>
    <h3>University of Toronto</h3>
    <h3>Department of Mechanical and Industrial Engineering</h3>
    <h3>MIE368 Analytics in Action </h3>
    <h3>(Fall 2020)</h3>
    <hr>
    <h1>Lab 4: Predict then Optimize </h1>
    <h3>October 21, 2020</h3>
</center>


# Introduction
In this lab we investigate how we can combine prediction models (predictive analytics) with optimization models (prescriptive analytics). The power of prescriptive analytics is that we can begin to influence decision-making! In this lab, we will look at a real world application of combining these two approaches. 

There are two high level modelling choices we can follow when designing a predict-then-optimize pipeline:


1.   Use the $\beta$ values from a model such as linear regression as coefficients in an optimization model
2.   Use the actual predicted values, i.e. $\hat{y}$, from a model like logistic regression as coefficients in an optimization model






### Exercises

1. Concisely explain the difference between predictive and prescriptive analytics.

___
__Question 1 Answer__:
Predictive analytics just seeks to determine what may happen in the future, while prescriptive analytics seeks to determine what decisions should be made to best guarantee that desired future events take place. 

___

2. You create a model that *determines how much traffic will be present during your drive to work.* Would that model fall under predictive or prescriptive analytics? Why?

___
__Question 2 Answer__:
This would fall under predictive analytics as we are just trying to predict traffic levels, not make any explicit decisions.

___

3. You create a model that *determines the best route to take to work by avoiding traffic*. Would that model fall under predictive or prescriptive analytics? Why? 

___
__Question 3 Answer__:
This would fall under prescriptive analytics as we are trying to influence optimal decision making.

___

# Application: Wheelchair Rugby (WCR)

In this lab, we will use a dataset describing Canada's paralympic wheelchair rugby team. It became an official paralympic sport in Sydney in 2000, and was originally called "*murder ball*"! It is highly recommended that you watch the following short 2 minute video going over the rules of wheelchair rugby. This should provide a good understanding for the lab, while also being quite interesting :)


<a href="https://www.youtube.com/watch?v=tSzFmlWgVsM&feature=emb_logo&ab_channel=ParalympicGames" target=""><img src="http://i3.ytimg.com/vi/tSzFmlWgVsM/hqdefault.jpg" 
alt="" width="240" height="180" border="10" /></a>


There are two main features to wheelchair rugby that are important to highlight:


1.   Each player on the team will have a phyical ability rating of one of the following values: {3.5, 3.0, 2.5, 2.0, 1.5, 1.0, 0.5, 0.0}. Note, this takes into account men's and women's slightly different rating systems. The higher the rating, the more mobility they have with their body.
2.   Teams must have four players on the court at any time. The sum of the physical ability ratings of those players on the court cannot exceed 8. 





### Exercises

1. Based on the video, which physical rating is more likely to be the goal scorers on a wheelchair rugby team?

  ___
 __Question 1 Answer__:

Players with a higher physical rating.
___

The second important feature we highlighted above should remind us of an optimization concept we learned recently: knapsack constraints. At first glance, it seems like there is room to gain a competitive advantage by building optimal lineups that maximize value while adhering to this enforced knapsack constraint. 

What exactly is the value/metric that we should try to optimize though? Lets do a quick review on *adjusted plus/minus per 100 possessions*, a metric frequently used in sports statistics. 

# Adjusted Plus/Minus per 100 possessions ($APM_{100}$)

The adjusted plus/minus per 100 possessions ($APM_{100}$) is a metric that measures how much impact a particular player has in the game. There are two usages of the term $APM_{100}$ that is important to differentiate:

1.   $APM_{100}$ for a ***stint***: which is the score differential divided by the number of possession for a particular stint multiplied by 100.
2.   $APM_{100}$ for a ***player***: which is a players contribution to achieving the $APM_{100}$ for a stint.

Note, a stint is defined as a portion of a game in which the same players were on the court throughout, i.e., there were no substitutions.

For example, lets say theres two teams, each of which had two players on the court for a particular stint. Lets call the players A1, A2, B1 and B2. Lets say the scoring for that stint was 11-5 in favour of team A, over 15 possessions. Therefore, the $APM_{100}$ for that **stint** is $(11-5)/15 *100 = +40$.

What we are interested in is the $APM_{100}$ for each of the **players** to see how they contributed to that +40. To get these values, we solve the following regression equation:

$\beta_{A1} + \beta_{A2} + \beta_{B1} + \beta_{B2} + \beta_{0} = +40$

Where $\beta_i$ is the $APM_{100}$ for **player** $i$, and $\beta_0$ is just an intercept term.

We would have an equation like the one above for **every** stint in a game, and this way, get accurate measures of the $\beta$'s for each **player**, aka, their impact in the game! 

A more comprehensive explanation and intuition behind $APM_{100}$ is found in the appendix. Note, you will not be quizzed on $APM_{100}$.


# Exploratory Data Analysis

We now import and explore the stint data for the Canadian paralympic WCR team. Each row of the data describes one stint. There are indicators for which team Canada played against, as well as how many of each type of physical rated player was on the court for the home and away team. Lastly, the final column has the $APM_{100}$ value for that stint.

In [1]:
# Import packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
import cvxpy as cp


# Load dataset
df = pd.read_csv('https://docs.google.com/uc?export=download&id=1LfmtKomWZO_1gd2O751GqMrLfpi8pBFo') 
df.head() # prints the first 5 rows of the dataframe 


,Home,Away,Away_Argentina,Away_Brazil,Away_Chile,Away_Colombia,Away_Denmark,Away_France,Away_Great_Britain,Away_Japan,Away_Poland,Away_Sweden,Away_USA,home_3_5,home_3_0,home_2_5,home_2_0,home_1_5,home_1_0,home_0_5,home_0_0,away_3_5,away_3_0,away_2_5,away_2_0,away_1_5,away_1_0,away_0_5,away_0_0,APM_100
0,Canada,Poland,0,0,0,0,0,0,0,0,1,0,0,0,1,0,2,0,1,0,0,1,0,0,2,0,0,1,0,-16.666667
1,Canada,Poland,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,1,0,1,0,0,2,0,0,1,0,4.638615
2,Canada,Poland,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,1,0,1,0,0,2,0,0,1,0,4.253957
3,Canada,Poland,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,1,0,0,0,0,4,0,0,0,0,0.557469
4,Canada,Poland,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,1,0,0,0,0,4,0,0,0,0,-5.970548


The table below contains the data dictionary.

|Feature          |Definition                                             |
|:---------------:|:------------------------------------------------------|
|Home           | The home team (always Canada)                  |
|Away            |The away team Canada played against |    
|Away_Argentina      |1 if the away team was Argentina, 0 o/w |
|...             |              | 
|Away_USA    |1 if the away team was USA, 0 o/w |
|home_3_5  |The number of 3.5 rated players on the court for the home team                        
|...             |              | 
|home_0_0  |The number of 0.0 rated players on the court for the home team
|away_3_5  |The number of 3.5 rated players on the court for the away team                        
|...             |              | 
|away_0_0  |The number of 0.0 rated players on the court for the away team
|APM_100 | $APM_{100}$ for that stint 



### Exercises

1. How many stints are in the dataset?

In [2]:
# Write your code here.  

# -------------------

df.shape[0]

# -------------------

334

2. How many stints did Canada play against each country?

In [3]:
# Write your code here.  

# -------------------

df['Away'].value_counts()

# -------------------

Japan            62
France           54
Denmark          47
USA              45
Colombia         28
Sweden           23
Brazil           21
Chile            15
Poland           13
Argentina        13
Great_Britain    13
Name: Away, dtype: int64

3. What is the average $APM_{100}$ over all stints?

In [4]:
# Write your code here.  

# -------------------

df['APM_100'].mean()

# -------------------

6.018087546661678

4. What is the average $APM_{100}$ for each country Canada faced?




In [5]:
# Write your code here.  

# -------------------

df.groupby('Away')['APM_100'].mean()

# -------------------

Away
Argentina        38.252628
Brazil            9.450532
Chile            27.255230
Colombia         11.768853
Denmark           0.457460
France           -3.333349
Great_Britain   -15.351736
Japan             7.283580
Poland           -1.021517
Sweden           32.226771
USA              -5.455680
Name: APM_100, dtype: float64

5. Based on the average $APM_{100}$ by country, which country does Canada perform the best against? What about the worst against?

___
__Question 5 Answer__:

It seems that Canada performs best against Argentina (average $APM_{100}$ = 38.25) and worst against Great_Britain (average $APM_{100}$ = -15.35).
___

# Phase One: Prediction

We now begin the first phase of our predict-then-optimize framework. We will use the loaded data to regress on the $APM_{100}$ for each stint. Since we don't have data on the actual players on the court, we'll use the available data to learn the $\beta$ values for each physical ability rating. 

To solve for the $\beta$ values in $APM_{100}$ calculations, it is common practice to use Ridge regression (i.e., L-2 Regularization, see Lab One for more information). If you are curious as to why we choose Ridge regression, the last few sections of [this blog post](https://squared2020.com/2017/09/18/deep-dive-on-regularized-adjusted-plus-minus-i-introductory-example/) provides the rationale. 

Note that we will face two major limitations. First, models that use $APM_{100}$ generally suffer from multicollinearity. We will ignore that issue for the sake of brevity, however, it is something you should consider more seriously in your projects. Second, we are violating the general rule of thumb that for a dataset with $N$ samples, we should use no more than $\sqrt{N}$ features. This is the reality of building models with real world data, it is never perfect!


The following code splits the data into a training and testing set and builds the model to estimate the $\beta$ values based on the home and away player's physical ratings.

In [6]:
# from sklearn.linear_model import RidgeCV
# from sklearn.model_selection import train_test_split

# Split the data into the training and testing set
X_train, X_test, y_train, y_test = train_test_split(df.drop('APM_100',1),  # all X data
                                                    df['APM_100'],  # All y data
                                                    test_size=0.30,  # Fraction of data in test set
                                                    shuffle=True,  # Randomly splits the data
                                                    random_state = 3  # Sets random seed for reproducability 
                                                    )

# Only look at the home and away physical ratings as dependent features
X_train = X_train.loc[:, 'home_3_5':'away_0_0']
X_test = X_test.loc[:, 'home_3_5':'away_0_0']

# Initialize and fit the cross-validated Ridge model
ridge = RidgeCV()
ridge.fit(X_train, y_train)

# Score the model
train_score = ridge.score(X_train, y_train)
test_score = ridge.score(X_test, y_test)
print(f'The train score is {train_score:.3f} and the test score is {test_score:.3f}')


The train score is 0.311 and the test score is 0.176


### Exercises

The training and testing score for the above model performs very poorly. Thinking back to our EDA, we discovered that the $APM_{100}$ greatly varies depending on which team Canada was playing against. Since we don't have data for individual players, which would be ideal, lets incorporate the away teams into the model. This should allow us to determine a $\beta$ for each physical rating, but scaled to that particular away team. 

1. Fill in the following code block to build a `RidgeCV` model that uses the columns from `'Away_Argentina':'away_0_0'` and call the model `ridge_teams`. Report the training and testing score. 

In [7]:
# Split the data into the training and testing set
X_train, X_test, y_train, y_test = train_test_split(df.drop('APM_100',1), df['APM_100'], test_size=0.30, shuffle=True, random_state = 3)

# Write your code here. 

# -------------------

X_train = X_train.loc[:, 'Away_Argentina':'away_0_0']
X_test = X_test.loc[:, 'Away_Argentina':'away_0_0']

# Initialize and fit the cross-validated Ridge model
ridge_teams = RidgeCV()
ridge_teams.fit(X_train, y_train)

# Score the model
train_score = ridge_teams.score(X_train, y_train)
test_score = ridge_teams.score(X_test, y_test)
print(f'The train score is {train_score:.3f} and the test score is {test_score:.3f}')

# -------------------


The train score is 0.862 and the test score is 0.830


This model performs much better! Lets take a look at the computed $\beta$'s to gain further insight. 

2. Print out the $\beta$'s for the model using `coef_` and `intercept_` attributes, stored in out `ridge_teams` model.

In [8]:
# Write your code here.  

# -------------------

betas = pd.Series(ridge_teams.coef_, index=X_train.columns)
betas = betas.append(pd.Series({"Intercept": ridge_teams.intercept_}))
print(betas)

# -------------------

Away_Argentina        30.293304
Away_Brazil           -0.279836
Away_Chile            27.849132
Away_Colombia          1.108798
Away_Denmark         -10.020181
Away_France          -11.689986
Away_Great_Britain   -22.734909
Away_Japan            -7.886676
Away_Poland          -20.489744
Away_Sweden           28.733298
Away_USA             -14.883201
home_3_5              41.583884
home_3_0              24.072339
home_2_5              17.103739
home_2_0               5.443597
home_1_5             -10.411467
home_1_0             -13.110144
home_0_5             -30.804578
home_0_0             -33.877371
away_3_5              -1.221446
away_3_0               0.313877
away_2_5              -3.499498
away_2_0               2.394348
away_1_5              -1.145152
away_1_0              -4.184837
away_0_5              -1.042160
away_0_0               8.384869
Intercept             -8.053351
dtype: float64


3. Based on the $\beta$'s for the away teams, which team does Canada perform the best against, and which one do they perform the worst against? Do these values match our insight found in the above EDA? 

___
__Question 3 Answer__:

It seems that Canada performs best against Argentina ($\beta_{Away\_Argentina} = +30.29$) and worst against Great_Britain ($\beta_{Away\_Great\_Britain} = -22.73$). This matches what we discovered in our EDA above.
___

4. Looking at just the $\beta$'s for the home player physical ratings, which rating is the best for Canada in terms of $APM_{100}$? Which rating is the worst?

___
__Question 4 Answer__:

Canada's best performing physical rating is home_3_5 ($\beta_{home\_3\_5} = +41.58$) and worst is home_0_0 ($\beta_{home\_0\_0} = -33.88$).
___

Now that we have a strongly fit model and $\beta$ coefficients, what should we do with them? Lets pretend that their was no knapsack constraint limiting the sum of the physical ratings of the players on the court to 8. Which four players would we always choose to play? We can easily say we'd always play 3.5 rated players, as they have the highest $\beta$. However, in reality, we cannot ignore this knapsack constraint, and this is where optimization comes in handy. Lets start using these $\beta$'s to build optimal lineups!  

# Phase Two: Optimization

We will now use the outputs derived from our prediction model, namely the $\beta$'s, as inputs in optimization models. Remember, the $\beta_i$'s can be seen as the value that players with physical rating $i$ provide to the game. We will build two different models, each with a different goal in mind. 



## IP Knapsack Model: Optimal Lineups

We formulate the model as a simple knapsack problem. The model will choose the number of players for every rating that we should put on the court in order to maximize our objective (i.e., maximize $APM_{100}$) while adhering to the knapsack constraint (i.e., the sum of physical rating $\leq8$). 

We define the integer decision variable, $x_i$, as the number of players on the court with rating $i$, $\forall\ i \in \{3.5, 3.0, 2.5, 2.0, 1.5, 1.0, 0.5, 0.0\}.$ 

We also know about the following parameters. Let $R_i$ be the physical rating of a player with rating $i$. Lastly, we incorporate the outputs of the prediction model, namely, we let $\beta_i$ be the $APM_{100}$ for home players with physical rating $i$. Putting it all together, the model is as follows:

\begin{aligned}
        \max  & \sum_{i} \beta_i x_i  \\
        s.t.  & \sum_{i} R_ix_i \leq 8\\
              & \sum_{i} x_i = 4\\
              & x_i \in \mathbb{Z} \quad \forall i\\
\end{aligned}

Why don't we include any decision variables based on the away_team, or the physical ratings of players on the away team? The general tip when it comes to optimization models, and prescriptive analytics as a whole, is it is best to put ourselves into the position of the key decision maker of the problem. In this context, we are thinking through the lens of the head coach of Canada's WCR team. Since we can only make decisions about what our team does, we do not include any decision variables about the away team. By maximizing what our players do, we will be maximizing over the set of decisions we actually have control over. 

We formulate this optimization model with cvxpy in the following code block.

In [9]:

# Vector for the physical ratings and beta's
ratings = [3.5, 3.0, 2.5, 2.0, 1.5, 1.0, 0.5, 0.0]
betas = betas['home_3_5':'home_0_0']

# Define the variable of how many players of each physical rating we choose
x = cp.Variable(len(ratings), integer=True)

# Create the objective of maximizing APM_100
obj = cp.Maximize(x*betas) 

# Create the constraints
cons = []
cons.append(x*ratings <= 8) # knapsack constraint on max physical rating sum
cons.append(sum(x) == 4)    # 4 players on the court
cons.append(x >= 0)         # non-negativity of x

# Solve the model
prob = cp.Problem(obj,cons)
prob.solve()

# Print the objective value
prob.solve(verbose=False)  # verbose = True allows you to see the solution process
print('The maximum APM_100 is {:.3f}'.format(obj.value))  # gives you the objective value

#Extract the optimal starting lineup
x_np_array = x.value.astype(int)  # extract the x values as a np array
x_values = pd.Series(x_np_array, index = betas.index)  # convert the np array to a series
print('The optimal lineup is: \n', x_values)


The maximum APM_100 is 36.180
The optimal lineup is: 
 home_3_5    2
home_3_0    0
home_2_5    0
home_2_0    0
home_1_5    0
home_1_0    1
home_0_5    0
home_0_0    1
dtype: int64


### Exercises

There is one issue with this model, it assumes we have as many players with each physical rating that we could want. Let us constrain this problem knowing that we only have the following breakdown of 12 player on our roster to choose from.


|Physical Rating | # of Players on Roster ($P$)|
|:---------------:|:--------------------:|
| 3.5    | 1   |          
| 3.0    | 2   |          
| 2.5    | 0   |          
| 2.0    | 2   |          
| 1.5    | 2   |           
| 1.0    | 1   |           
| 0.5    | 3   |           
| 0.0    | 1   |     

Therefore, we need to add the following constraints to the model: $ \quad  x_i \leq P_i \quad \forall i $

1. Add this constraint to the model by `cons.append()` and re-solve for the optimal objective value and optimal lineup.

In [10]:
# The following vector represents how many players of each physical rating we have on our roster
# Use this vector in your new constraint.
roster = [1,2,0,2,2,1,3,1]

# Write your code here.  

# -------------------

# Add the new roster constraint
cons.append(x <= roster)

# Solve the model
prob = cp.Problem(obj,cons)
prob.solve()

# Print the objective value
prob.solve(verbose=False)  # verbose = True allows you to see the solution process
print('The new maximum APM_100 is {:.3f}'.format(obj.value))  # gives you the objective value

#Extract the optimal starting lineup
x_np_array = x.value.astype(int)  # extract the x values as a np array
x_values = pd.Series(x_np_array, index = betas.index)  # convert the np array to a series
print('The new optimal lineup is: \n', x_values)

# -------------------


The new maximum APM_100 is 23.506
The new optimal lineup is: 
 home_3_5    1
home_3_0    0
home_2_5    0
home_2_0    1
home_1_5    1
home_1_0    1
home_0_5    0
home_0_0    0
dtype: int64


We just solved for the optimal lineup that will maximize our $APM_{100}$. However, there is a large drawback to this approach, namely, this lineup couldn't possibly play for the entire 32 minutes, so we'll need to incorporate substitutions! This makes the problem more nuanced but also more useful.

Lets state a simplifying assumption that we will only make a substitution at the beginning of each quarter, and no players from the previous quarter can play in the current quarter. With this information, we can state that the lineup we solved above would be the optimal lineup we would choose for the *1st quarter*, i.e., the *starting lineup*.

2. What is the objective value and optimal lineup for the 2nd quarter? Note: CVXPY doesn't let you modify/delete old constraints so you will have to reformulate an entirely new model. Add "_2Q" to all your model definitions, i.e., `x_2Q`, `obj_2Q`, `cons_2Q` and `prob_2Q`.  
(Hint: you will also have to modify your `roster` vector.)

In [11]:
# Write your code here.  

# -------------------

# adjusted roster, subtracting a 3.5, 2.0, 1.5 and 1.0 rated player from our 1st quarter lineup
roster_2Q = roster - x_np_array


# Define the variable of how many players of each physical rating we choose
x_2Q = cp.Variable(len(ratings), integer=True)

# Create the objective of maximizing APM_100
obj_2Q = cp.Maximize(x_2Q*betas) 

# Create the constraints
cons_2Q = []
cons_2Q.append(x_2Q*ratings <= 8) # knapsack constraint on max physical rating sum
cons_2Q.append(sum(x_2Q) == 4)    # 4 players on the court
cons_2Q.append(x_2Q >= 0)         # non-negativity of x
cons_2Q.append(x_2Q <= roster_2Q)    # new roster constraint for the 2nd quarter


# Solve the model
prob_2Q = cp.Problem(obj_2Q,cons_2Q)
prob_2Q.solve()

# Print the objective value
prob.solve(verbose=False)  # verbose = True allows you to see the solution process
print('The maximum APM_100 for the 2nd quarter is {:.3f}'.format(obj_2Q.value))  # gives you the objective value

#Extract the optimal starting lineup
x_2Q_np_array = x_2Q.value.astype(int)  # extract the x values as a np array
x_2Q_values = pd.Series(x_2Q_np_array, index = betas.index)  # convert the np array to a series
print('The optimal lineup for the 2nd quarter is: \n', x_2Q_values)

The maximum APM_100 for the 2nd quarter is 19.711
The optimal lineup for the 2nd quarter is: 
 home_3_5    0
home_3_0    2
home_2_5    0
home_2_0    1
home_1_5    0
home_1_0    0
home_0_5    0
home_0_0    1
dtype: int64


3. Lets suppose we only wanted to substitute two players when we begin the 2nd quarter. What is one approach to determine the best two players to swap? You don't need to code up this answer, just provide some discussion.

___
__Question 3 Answer__:

A brute force approach would be to try all the combinations of subbing out two players and building the new lineup, and picking the substitution decision that led to the best new objective value. There would be ${4 \choose 2} = 6$ times we'd have to run the model.
___


Lets say that now we want to make the optimal lineup for the 3rd quarter without using any players for the 2nd quarter. If we were to adjust our roster accordingly, our model would once again choose the original 4 players from the 1st quarter to play in the 3rd quarter. (Similarly, the 2nd quarter lineup would play in the 4th quarter). What crucial component of a player's reality are we neglecting here?

We are not accounting for player fatigue and how that will impact a player's value/$APM_{100}$. We need a way to model the decrease in a player's acheivable APM as a function of their total playing time. Lets explore that with our next optimization model!


## LP Model: Optimal Play Time Allocation (Theory)

Lets try to develop a model that will make some assumptions on how a player's $APM_{100}$ deteriorates as a function of their playing time and see if we can create optimal playing time allocations for each player. Its important to understand how the results of this model will be utilized, namely, it will tell us how many total minutes each player should play. This doesn't tell us anything about what lineup they should be on or when they should be on the court. We leave it up to the coach to design their lineups while adhering to this maximum time limit that this model will recommend. 

This model will be a bit more complicated, so take your time understanding each of the following pieces. We skip certain details of this formulation in the main body of the lab, but the full discussion is found in the appendix.

The following function (derived fully in the appendix), $C(t)$, represents how much value, or $APM_{100}$, a player will achieve by playing for a total time $t$. 

Its important to notice the decreasing slope as the time $t$ increases. This can intuitively be understood that as a player begins to fatigue, their ability to earn value per minute of playing time is decreasing. 

\begin{aligned}
  C(t) =
  \begin{cases}
          \beta t  && 0 \leq t < 6 \\
          0.75\beta t + 1.5\beta  && 6 \leq t < 12 \\
          0.50\beta t + 4.5\beta && 12 \leq t < 24 \\
          0.25\beta t + 10.5\beta  && 24 \leq t \leq 32 \\
  \end{cases}
\end{aligned}

We can plot this piece-wise function, $C(t)$, below. Note that the bolded sections of the plot is what we are most concerned with, as these sections are where each $y = mt + b$ line falls under its appropriate interval. 

<figure>
<img src="https://docs.google.com/uc?export=view&id=1izWlZtip_51Q6PBhufB1sdaWQKEMEQ_W"
alt="" />
<figcaption>Relationship between Cumulative APM and Playing Time with a $\beta = 12$.</figcaption>
</figure>

Note, this function takes the form of a **concave** piece-wise linear function due to its concavity. These types of functions have a special property that allows us to write them as:

\begin{aligned}
C(t) = \min_{i = i,...,k} \{m_it + b_i\}
\end{aligned}


Where $k$ equals the number of "segments" of the function. In our case, $k=4$. Also noting that the $m$'s and the $b$'s are the slopes and intercepts of each line segment.

Why is it a minimum over each segment? Lets take a look at what $C(t=3)$ would be. Looking at the grey dotted line in the graph above, this line intersects each of the 4 line segments at different $y$ values. However, for the interval of $[0,6]$, we should only look at the red, bolded segment. By choosing the minumum of these 4 intersection points, we're gauranteed to stay on the correct segment for each $t$, depending on which interval $t$ lies on! 

We run into an issue with the players who have a negative $\beta$. For this, we define the following concace piece-wise function to model how their $APM_{100}$ gets more negative, i.e., *worse* over time (more discussion on this in the appendix):

\begin{aligned}
  D(t) =
  \begin{cases}
          \beta t  && 0 \leq t < 6 \\
          1.25\beta t - 1.5\beta  && 6 \leq t < 12 \\
          1.50\beta t - 4.5\beta && 12 \leq t < 24 \\
          1.75\beta t - 10.5\beta  && 24 \leq t \leq 32 \\
  \end{cases}
\end{aligned}

These functions are also concave, so we can express $D(t)$ as: 

\begin{aligned}
D(t) = \min_{i = i,...,k} \{c_it + d_i\}
\end{aligned}

---------

Lets start defining our model now!

First lets define our decision variable $t_i$ as the number of minutes player $i$ will play in the game. We want to $\max C_i(t_i)$ or $D_i(t_i)$ for each player depending on the sign of their $\beta$ value. Lets define the players with a positive $\beta$ as belonging to the set $P$ and those with negative $\beta$ as belonging to the set $N$. This results in the following objective function: $ \max \left( \sum_{i \in P} C_i(t_i) + \sum_{i \in N} D_i(t_i) \right)$. Exploiting the concave piece-wise form of these functions, we can transform $C_i(t_i)$ and $D_i(t_i)$ so the objective function becomes: 

\begin{aligned}
\max \left( \sum_{i \in P} \min_{k=1,2,3,4} (m^k_it_i + b^k_i) + \sum_{i \in N} \min_{k=1,2,3,4} (c^k_it_i + d^k_i) \right)
\end{aligned}

This current objective function isn't linear yet, so we can't use linear programming as is. However, we can define a new variable $Z_i$ which will capture the APM value for the players with positive $\beta$ and similarly define $Q_i$ for players with negative $\beta$. You don't need to understand how this transformation works, but this makes the model linear!. (If you are curious, we can explain it in the lab session). The objective function becomes 

\begin{aligned}
\max \left( \sum_{i \in P} Z_i + \sum_{i \in N} Q_i \right)
\end{aligned}

and we add the following constraints to "capture" the correct objective value:

\begin{aligned}
Z_i \leq (m^k_it_i + b^k_i) \quad \forall i \in P,k \\
Q_i \leq (c^k_it_i + d^k_i) \quad \forall i \in N,k \\
\end{aligned}

Lastly, we need constraints to keep $t$ nonnegative. We also need a constraint to ensure a player doesn't play for more time than the entire game ($32$ minutes). Lastly, the sum of all the playing times need to equal $128$, since there will always be $4$ players on the court, and $4*32 = 128$. Therefore, we add the following constraints:

\begin{aligned}
t_i \geq 0 \quad \forall i  \\
t_i \leq 32 \quad \forall i  \\
\sum_{i} t_i = 128 \\
\end{aligned}


Putting it all together, we have the following **linear** program:

\begin{aligned}
        \max & \left( \sum_{i \in P} Z_i + \sum_{i \in N} Q_i \right) \\
        s.t. \quad & Z_i \leq (m^k_it_i + d^k_i) \quad \forall i \in P,k \\
        & Q_i \leq (c^k_it_i + d^k_i) \quad \forall i \in N,k \\
        & t_i \geq 0 \quad \forall i \\
        & t_i \leq 32 \quad \forall i \\
        & \sum_{i} t_i = 128\\
\end{aligned}




## LP Model: Optimal Play Time Allocation (Coding)

Lets now code up this model! We'll start by defining `df_players`, which stores every players' rating and appropriate $\beta$ value.

In [12]:
# Create the player info df_players
player_dict = {'Player1': [3.5, betas['home_3_5']], 'Player2': [3.0, betas['home_3_0']],
               'Player3': [3.0, betas['home_3_0']], 'Player4': [2.0, betas['home_2_0']],
               'Player5': [2.0, betas['home_2_0']], 'Player6': [1.5, betas['home_1_5']], 
               'Player7': [1.5, betas['home_1_5']], 'Player8': [1.0, betas['home_1_0']],
               'Player9': [0.5, betas['home_0_5']], 'Player10': [0.5, betas['home_0_5']], 
               'Player11': [0.5, betas['home_0_5']], 'Player12': [0.0, betas['home_0_0']]}
df_players = pd.DataFrame(player_dict, index = ['rating', 'beta'])
df_players.head()

,Player1,Player2,Player3,Player4,Player5,Player6,Player7,Player8,Player9,Player10,Player11,Player12
rating,3.500000,3.000000,3.000000,2.000000,2.000000,1.500000,1.500000,1.000000,0.500000,0.500000,0.500000,0.000000
beta,41.583884,24.072339,24.072339,5.443597,5.443597,-10.411467,-10.411467,-13.110144,-30.804578,-30.804578,-30.804578,-33.877371


Next, we formulate the linear program

In [13]:
# We define the m, c and d vectors for the piecewise linear segments of the cumulative graphs
# We define these vectors independant of beta, since the beta value will differ for each player and can be linearly multipled on afterwards
m = [1, 0.75, 0.5, 0.25]
b = [0, 1.5, 4.5, 10.5]
c = [1, 1.25, 1.5, 1.75]
d = [0, -1.5, -4.5, -10.5]

# This indexes the players into positive or negative beta values
P = [0,1,2,3,4] # index of positive beta players
N = [5,6,7,8,9,10,11] # index of negative beta players
K = 4 # number of segments

# Define the variable of how many players of each physical rating we choose
x = cp.Variable(12)
Z = cp.Variable(len(P)) # only need it for positive beta players
Q = cp.Variable(len(N)) # only need it for negative beta players

# Create the objective of maximizing APM_100
obj = cp.Maximize(sum(Z) + sum(Q)) 

# Create the constraints
cons = []
cons.append(sum(x) == 128)    # Total sum of all players time is 128
cons.append(x >= 0)         # non-negativity of time played
cons.append(x <= 32)         # max play time per player

# Piecewise constraints to capture objective value for positive players
for i in P:
  for k in range(K):
    cons.append(Z[i] <= df_players.loc['beta'][i] * (m[k]*x[i] + b[k]))

# Piecewise constraints to capture objective value for negative players
for i in N:
  for k in range(K):
    cons.append(Q[i-len(P)] <= df_players.loc['beta'][i] * (c[k]*x[i] + d[k]))

# Solve the model
prob = cp.Problem(obj,cons)
prob.solve()

# Print the objective value
prob.solve(verbose=False)  # verbose = True allows you to see the solution process
print('The maximum APM_100 is {:.3f}'.format(obj.value))  # gives you the objective value

#Extract the optimal playing time allocation
x_np_array = x.value.astype(float)  # extract the x values as a np array
x_values = pd.Series(np.round(x_np_array,2), index = df_players.columns)  # convert the np array to a series
print('The optimal playing time allocation per player is: \n', x_values)

The maximum APM_100 is 1796.068
The optimal playing time allocation per player is: 
 Player1     32.0
Player2     32.0
Player3     32.0
Player4     16.0
Player5     16.0
Player6     -0.0
Player7     -0.0
Player8     -0.0
Player9      0.0
Player10     0.0
Player11     0.0
Player12     0.0
dtype: float64


### Exercises

1. Based on the optimal play-time allocation solution above, how many minutes should player 1 play? How about players 5 and 10?

___
__Question 1 Answer__:

Player 1 = 32 minutes.
Player 5 = 16 minutes.
Player 10 = 0 minutes.
___

2. What do you notice about the playing time allocation for players of the same physical rating?

___
__Question 2 Answer__:

They are all assigned the same playing time.
___

3. Is it reasonable that every player has the same interval length for each fatigue threshold? If not, what would be more realistic?

___
__Question 3 Answer__:

Its unlikely that people fatigue at the same time threshold. We imagine a higher rated $\beta$ player to be able to last longer at that level before their decline. This would make the model more intersting!
___

4. By not explicitly considering the knapsack constraint, our current model suggests play time allocations that won't allow us to make feasible lineups. This is because the model is essentially saying to only play the best rated players, which makes it impossible to create lineups that have a physical rating $\leq 8$. What are some constraints we can add that will result in more feasible play time allocations?

___
__Question 4 Answer__:

There are two ideas we can readily implement:


*   We can put a minimum playing time on every player, that way they are forced to play which can lead to feasible lineups
*   We can enforce certain players to always play together. For example, adding a constraint that says whenever a 3.5 plays, a 0.5 also plays. Similarly for a 3.0 and 1.0.

___

## LP Model: Optimal Play Time Allocation (More Feasible Model)

As we may have realized, this optimal play time allocation model doesn't take into account the knapsack constraint. This leads to only assigning playing times to the highly rated players, which would lead to infeasible lineups. We can add some extra constraints that will enforce the optimal play time allocations to be more feasible. 

We note that historically, 3.5 rated players will be on the same lineup as 0.5 rated players. Similarly, 3.0 rated players are usually with 1.0 players. Lets add a constraint that says the playing time among players of these ratings are equal.

### Exercises

1. Use the `cons.append()` feature to add two constraints. The first making the time for the 3.5 rated players equal the total time of all the 0.5 rated players. Note, player 1 is 3.5 and players 9-11 are 0.5. The second constraint should be similar, but for the 3.0 and 1.0 rated players. Solve the model and print the optimal lineup.

In [14]:
# Write your code here.  

# -------------------

cons.append(x[0] == x[8] + x[9] + x[10])  # equal playing time for 3.5 and 0.5 players
cons.append(x[1] + x[2] == x[7] )  # equal playing time for 3.0 and 1.0 players

# Solve the model
prob = cp.Problem(obj,cons)
prob.solve()

# Print the objective value
prob.solve(verbose=False)  # verbose = True allows you to see the solution process
print('The maximum APM_100 is {:.3f}'.format(obj.value))  # gives you the objective value

#Extract the optimal playing time allocation
x_np_array = x.value.astype(float)  # extract the x values as a np array
x_values = pd.Series(np.round(x_np_array,2), index = df_players.columns)  # convert the np array to a series
print('The optimal playing time allocation per player is: \n', x_values)

# -------------------

The maximum APM_100 is 367.382
The optimal playing time allocation per player is: 
 Player1     12.0
Player2     10.0
Player3     10.0
Player4     32.0
Player5     32.0
Player6      0.0
Player7      0.0
Player8     20.0
Player9      4.0
Player10     4.0
Player11     4.0
Player12     0.0
dtype: float64


2. We notice that some players get no playing time at all. Lets enforce another constraint that each player has to play at least 4 minutes. Add that constraint and re-solve the model and report the optimal lineup.

In [15]:
# Write your code here.  

# -------------------

cons.append(x >= 4)     # Each player plays at least 4 minutes

# Solve the model
prob = cp.Problem(obj,cons)
prob.solve()

# Print the objective value
prob.solve(verbose=False)  # verbose = True allows you to see the solution process
print('The maximum APM_100 is {:.3f}'.format(obj.value))  # gives you the objective value

#Extract the optimal playing time allocation
x_np_array = x.value.astype(float)  # extract the x values as a np array
x_values = pd.Series(np.round(x_np_array,2), index = df_players.columns)  # convert the np array to a series
print('The optimal playing time allocation per player is: \n', x_values)

# -------------------

The maximum APM_100 is 158.247
The optimal playing time allocation per player is: 
 Player1     12.0
Player2      7.0
Player3      7.0
Player4     32.0
Player5     32.0
Player6      4.0
Player7      4.0
Player8     14.0
Player9      4.0
Player10     4.0
Player11     4.0
Player12     4.0
dtype: float64


3. These are our results based on our computed $\beta$'s from the prediction model from phase one. Lets reformulate an entirely new model with new $\beta$'s defined in `df_players_new`. You'll have to completely redefine the entire model and include all the constraints we added from exercises 1 and 2 because of how CVXPY works. Make sure to change your $P$ and $N$ sets. Re-solve and report the optimal lineup.

In [16]:
# Use this df_players_new
player_dict_new = {'Player1': [3.5, 25], 'Player2': [3.0, 21], 'Player3': [3.0, 21], 'Player4': [2.0, 14],
               'Player5': [2.0, 14], 'Player6': [1.5, 8], 'Player7': [1.5, 8], 'Player8': [1.0, 2],
               'Player9': [0.5, -2], 'Player10': [0.5, -2], 'Player11': [0.5, -2], 'Player12': [0.0, -7]}
df_players_new = pd.DataFrame(player_dict_new, index = ['rating', 'beta'])

# Write your code here.  

# -------------------

m = [1, 0.75, 0.5, 0.25]
b = [0, 1.5, 4.5, 10.5]
c = [1, 1.25, 1.5, 1.75]
d = [0, -1.5, -4.5, -10.5]

# This indexes the players into positive or negative beta values
P = [0,1,2,3,4,5,6,7] # index of positive beta players
N = [8,9,10,11] # index of negative beta players
K = 4

# Define the variable of how many players of each physical rating we choose
x = cp.Variable(12)
Z = cp.Variable(len(P)) # only need it for positive beta players
Q = cp.Variable(len(N)) # only need it for negative beta players

# Create the objective of maximizing APM_100
obj = cp.Maximize(sum(Z) + sum(Q)) 

# Create the constraints
cons = []
cons.append(sum(x) == 128)    # Total sum of all players time is 128
cons.append(x >= 0)         # non-negativity of time played
cons.append(x <= 32)         # max play time per player

cons.append(x[0] == x[8] + x[9] + x[10]) # keep time equal for 3.5 and 0.5 players
cons.append(x[1] + x[2] == x[7] )        # keep time equal for 3.0 and 1.0 players
cons.append(x >= 4)                      # neach player gets at least 4 minutes


# Piecewise constraints to capture objective value for positive players
for i in P:
  for k in range(K):
    cons.append(Z[i] <= df_players_new.loc['beta'][i] * (m[k]*x[i] + b[k]))

# Piecewise constraints to capture objective value for negative players
for i in N:
  for k in range(K):
    cons.append(Q[i-len(P)] <= df_players_new.loc['beta'][i] * (c[k]*x[i] + d[k]))


# Solve the model
prob = cp.Problem(obj,cons)
prob.solve()

# Print the objective value
prob.solve(verbose=False)  # verbose = True allows you to see the solution process
print('The maximum APM_100 is {:.3f}'.format(obj.value))  # gives you the objective value

#Extract the optimal playing time allocation
x_np_array = x.value.astype(float)  # extract the x values as a np array
x_values = pd.Series(np.round(x_np_array,2), index = df_players_new.columns)  # convert the np array to a series
print('The optimal playing time allocation per player is: \n', x_values)

# -------------------

The maximum APM_100 is 1186.500
The optimal playing time allocation per player is: 
 Player1     12.0
Player2     12.0
Player3     12.0
Player4     20.0
Player5     20.0
Player6      6.0
Player7      6.0
Player8     24.0
Player9      4.0
Player10     4.0
Player11     4.0
Player12     4.0
dtype: float64


4. Did the optimal play time allocation change? Why do you think it did?

___
__Question 4 Answer__:

With the new $\beta$ values, the value of each player has shifted, so the geometry of the piece-wise linear functions have changed. This means, that if we introduce better or worse players into our roster, everyones optimal play time allocation will change.

___

# Reflection
As this is an _Analytics in Action_ course, we should think about how valuable our models are with respect to real-world applications. Position yourself as WCR coach or analytics staff and answer the following questions.

1. What is the limitation of calculating $\beta$'s for physical ratings, instead of individual players

___
__Question 1 Answer__:

Players of the same physical rating can perform drastically differently. For example, maybe the average $\beta_{3.5} = +30$, but we could have two players, say Bob and Jill, who could have very different levels of skill with a $\beta_{bob} = +21$ and a $\beta_{jill} = +39$.  
___

2. What other ways could we include fatigue in the model that is more realistic than our concave piece-wise linear function?

___
__Question 2 Answer__:

We could include how performance would fluctuate in and out of the intermissions and timeouts. For example, the performance of a player who played the 1st and 4th quarter, would be different than a player who played the 2nd and 4th quarters consecutively. Even though they played the same number of minutes, the concentration of those minutes vastly differ.
___

3. Can you come up with one way in which the knapsack model and feasible play time allocation model could be used together? i.e. the results of one model would influence the other, making both of them more useful.

___
__Question 3 Answer__:

One way to incorporate the models is we run the initial knapsack model to get the starting lineup. Then, as players in that lineup hit their maximum time allocation, we can take them off the roster, and reoptimize the knapsack model to find the second lineup. We can repeat this every time someone's time runs up!
___

# Appendix A: Adjusted Plus/Minus (APM)

Here are some important terms to know to help understand [adjusted plus/minus](https://www.agilesportsanalytics.com/sports-analytics-adjusted-plus-minus/#:~:text=Adjusted%20Plus%20Minus%20(APM)%20is,technique%20used%20for%20evaluating%20players.&text=The%20%2B%2F%2D%20stat%20is%20the,player%20was%20on%20the%20floor.) (APM) (note, you are not expected to know adjusted plus/minus for the quiz):


*   **Stint:** A stint is a consecutive amount of playing time in which the same players on the court were present throughout (i.e. no substitutions were made). This allows us to analyze what took place in the game for that particular set of players (home and away). 
*   **Regular Plus/Minus:** For each stint, we can calculate the score differential, and assign that value to each player on the court for that stint. (We assign the negated value for the away players). To get a player's final plus/minus, we add up all their individual plus/minus values for each stint they were in. This approach, while useful, doesn't take into account player interactions. 
*   **Adjusted Plus/Minus:** Uses regression to determine which players in each stint most impacted the game. We don't simply add up the adjusted plus/minus values for each stint a particular player was in. We'll go through the calculations below.
*   **Adjusted Plus/Minus per 100 Possesions:** Simply taking the adjusted plus/minus of a particular stint, dividing it by the number of possessions that took place during that stint, and multipling it by 100.

As an illustrative example, consider the hypothetical sport comprised of 3 players, with only two players per team allowed on the court at any time. We will identify the players as A1, A2, and A3 for Team A, and B1, B2,and B3 for Team B. Consider the following 3 stints.

*   A1, A2, vs. B1, B2 resulted in 11-5 over 15 possessions
*   A1, A3, vs. B2, B3 resulted in 4-9 over 7 possessions
*   A2, A3, vs. B1, B3 resulted in 8-10 over 13 possessions

Lets consider the regular plus minus for just player A2. The first time A2 was on the court, stint 1, the plus/minus was +6. The second time was in stint 3, in which the plus/minue was -2. Therefore, the overall plus/minus for A2 is +4. We can see that the problem with this approach is we can't tell if A2 was really the driving factor to earn that +4, or just happened to be on the court when better players were on as well. 

Adjusted plus/minus per 100 possession ($APM_{100}$) considers the system of linear equations that represents players playing on a court for each stint. There is one equation for each stint. The target variable, $Y_j$, is the $APM_{100}$ for stint $j$. The independent variables are binary indicators, $A_{i,j}$, set to $1$, if player $i$ was on the court during stint $j$, $0$ otherwise. We are trying to solve for the $\beta_i$'s, which is the $APM_{100}$ for player $i$. We also include a $\beta_0$ as an intercept for each equation. 

To summarize, there is an $APM_{100}$ value for the entire stint. We are trying to distribute that value to each player on the court, to see how they contributed towards it. Therefore, the $\beta$'s (aka the $APM_{100}$'s for each player) are additive, and sum up to the $APM_{100}$ of the entire stint. The following general system of equations can be use for our example: 

\begin{array}{ll}
Y_1 = \beta_0 + \beta_1A_{1,1} + \beta_2A_{2,1} + \beta_3A_{3,1} + \beta_4B_{1,1}+ \beta_5B_{2,1} + \beta_6B_{3,1}\\
Y_2 = \beta_0 + \beta_1A_{1,2} + \beta_2A_{2,2} + \beta_3A_{3,2} + \beta_4B_{1,2}+ \beta_5B_{2,2} + \beta_6B_{3,2}\\
Y_3 = \beta_0 + \beta_1A_{1,3} + \beta_2A_{2,3} + \beta_3A_{3,3} + \beta_4B_{1,3}+ \beta_5B_{2,3} + \beta_6B_{3,3}\\
\end{array}

Plugging in the exact values from the stint information, the equations boil down to the following (note $Y_1 = (11-5)/15 * 100 = +40$): 

\begin{array}{ll}
+40.00 = \beta_0 + \beta_1 + \beta_2 + \beta_4+ \beta_5\\
-71.43 = \beta_0 + \beta_1 + \beta_3 + \beta_5 + \beta_6\\
-15.38 = \beta_0 + \beta_2 + \beta_3 + \beta_4+ \beta_6\\
\end{array}

Solving for the $\beta$'s will give us the adjusted plus/minus per 100 possessions for each player. This will lead to a better understanding of how each individual player impacted the game!

# Appendix B: $APM_{100}$ and Fatigue


Lets assume that a players performance, or APM, starts off at their $\beta$ value and then has incremental drops each time they cross a particular threshold of total playing time according to the function $f(t)$, defined below. (If we wanted to be more precise, we could have done a time-sensitivity anaysis to precisely see how APM fluctuates over time, but this was out of scope for the lab).

\begin{aligned}
  f(t) =
  \begin{cases}
          \beta  && 0 \leq t < 6 \\
          0.75\beta && 6 \leq t < 12 \\
          0.5\beta && 12 \leq t < 24 \\
          0.25\beta && 24 \leq t \leq 32 \\
  \end{cases}
\end{aligned}

Lets take a look at what this plot would look like for a $\beta = 12$:

<figure>
<img src="https://docs.google.com/uc?export=view&id=13qAl8-HEUnqhCN2x4q4nGbYZ_p3y_y8c"
alt="" />
<figcaption>Relationship between APM and Playing Time with a $\beta = 12$.</figcaption>
</figure>

So now we know at what level they'll be playing at for each time $t$. But what we really want is their **cumulative** APM for their **total** playing time $t$. Going back to first year calculus, what we want is the area under the curve, so we need to integrate this piece-wise function. Luckily these are constant functions, so the integral is very easy to compute. We get the following function: 

\begin{aligned}
  C(t) =
  \begin{cases}
          \beta t  && 0 \leq t < 6 \\
          0.75\beta t + 1.5\beta  && 6 \leq t < 12 \\
          0.50\beta t + 4.5\beta && 12 \leq t < 24 \\
          0.25\beta t + 10.5\beta  && 24 \leq t \leq 32 \\
  \end{cases}
\end{aligned}

We can plot this piece-wise function, $C(t)$, below. Note that the bolded sections of the plot is what we are most concerned with, as these sections are where each $y = mt + b$ line falls under its appropriate interval. 

<figure>
<img src="https://docs.google.com/uc?export=view&id=1izWlZtip_51Q6PBhufB1sdaWQKEMEQ_W"
alt="" />
<figcaption>Relationship between Cumulative APM and Playing Time with a $\beta = 12$.</figcaption>
</figure>

Note, this function takes the form of a **concave** piece-wise linear function due to its concavity. These types of functions have a special property that allows us to write them as:

\begin{aligned}
C(t) = \min_{i = i,...,k} \{m_it + b_i\}
\end{aligned}

For players with negative $\beta$ values, we use the following piecewise function to highlight how their $APM_{100}$ gets more negative, aka, worse, over time.

\begin{aligned}
  g(t) =
  \begin{cases}
          \beta  && 0 \leq t < 6 \\
          1.25\beta && 6 \leq t < 12 \\
          1.5\beta && 12 \leq t < 24 \\
          1.75\beta && 24 \leq t \leq 32 \\
  \end{cases}
\end{aligned}

The cumulative function taken by integrating is: 

\begin{aligned}
  D(t) =
  \begin{cases}
          \beta t  && 0 \leq t < 6 \\
          1.25\beta t - 1.5\beta  && 6 \leq t < 12 \\
          1.50\beta t - 4.5\beta && 12 \leq t < 24 \\
          1.75\beta t - 10.5\beta  && 24 \leq t \leq 32 \\
  \end{cases}
\end{aligned}

These functions are also concave, so we can express $D(t)$ as: 

\begin{aligned}
D(t) = \min_{i = i,...,k} \{c_it + d_i\}
\end{aligned}

